In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.metrics import r2_score
from scipy.stats import norm
from sklearn.cluster import KMeans
import random
from statsmodels.regression.linear_model import OLS
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from gurobipy import *

# suppress all warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# number of items
n = 20

# number of features
d = 5

# number of observations for each item
obs = 10

# number of clusters
z = 2

# number of trials
T = 100

# noise
sigma = 1
# prob
p, q = 1/3, 1/3

In [4]:
def fit_models_upd(Ru,Rl):
    

    r2_alg_t, r2_b1_t, r2_b2_t, r2_lasso_t, r2_clus_t,r2_true_t =  np.zeros(T), np.zeros(T), np.zeros(T),np.zeros(T), np.zeros(T),np.zeros(T)
    
    
    for t in range(T):

        # generate data
        data = np.random.rand(n*obs,d)
        data_test = np.random.rand(int(n*obs/2),d)

        # generate clusters and betas
        feature_dict = {}
        clus_dict = {} # might not be used depending on the feature dict

        for i in range(d):
            # whether this feature should be estimated at the department level
            feature_dict[i] = np.random.choice(['dept', 'clus', 'sku'], p = [p, q, 1-p-q])


        items = set(range(n))

        for i in range(z):
            clus_items = random.sample(items,int(n/z))
            clus_dict[i] = clus_items
            items = items - set(clus_items)

        # generate Beta
        beta_dict = {}
        num_cols = 0
        for i in range(d):
            if feature_dict[i] == 'dept':
                beta_dict[i] = (np.random.random_sample() - 0.5) * 10
                num_cols += 1
            elif feature_dict[i] == 'clus':
                beta_dict[i] = (np.random.random_sample(size = z) - 0.5) * 10
                num_cols += z
            else:
                beta_dict[i] = (np.random.random_sample(size = n) - 0.5) * 10
                num_cols += n

        X_true = np.zeros((n*obs, num_cols))
        X_test = np.zeros((int(n*obs/2), num_cols))

        beta = np.zeros(num_cols)
        count = 0
        for i in range(d):
            if feature_dict[i] == 'dept':
                beta[count] = beta_dict[i]
                X_true[:,count] = data[:,i]
                X_test[:,count] = data_test[:,i]

                count += 1

            elif feature_dict[i] == 'clus':
                for j in range(z):
                    clus_items = clus_dict[j]
                    for sku in clus_items:
                        X_true[sku*obs:(sku+1)*obs, count] = data[sku*obs:(sku+1)*obs, i]
                        X_test[int(sku*obs/2):int((sku+1)*obs/2), count] = data_test[int(sku*obs/2):int((sku+1)*obs/2), i]

                    beta[count] = beta_dict[i][j]
                    count += 1
            else:
                for j in range(n):
                    X_true[j*obs:(j+1)*obs, count] = data[j*obs:(j+1)*obs, i]
                    ######
                    ######
                    ####### 
                    X_test[int(j*obs/2):int((j+1)*obs/2), count] = data_test[int(j*obs/2):int((j+1)*obs/2), i]
                    ######
                    ######
                    #######
                    beta[count] = beta_dict[i][j]
                    count += 1

        epsilon = np.random.multivariate_normal(np.zeros(n*obs), np.identity(n*obs)*sigma)
        epsilon_test = np.random.multivariate_normal(np.zeros(int(n*obs/2)), np.identity(int(n*obs/2))*sigma)
        y = np.dot(X_true, beta) + epsilon
        y_test = np.dot(X_test, beta) + epsilon_test
        

        # model fitting - b1
        y_pred = []
        ols_models = {}
        for i in range(n):
            model_i = OLS(y[i*obs:(i+1)*obs], data[i*obs:(i+1)*obs,:], hasconst = False)
            ols_models[i] = model_i.fit()
            y_pred += list(ols_models[i].predict(data_test[int(i*obs/2):int((i+1)*obs/2),:]))

        r2_b1_t[t] = r2_score(y_test, np.array(y_pred))
        


        # model fitting - our method
        aggre_level = []
        clus_columns = []
        all_coeff = np.zeros((n,d))
        all_coeff[0,:] = ols_models[0].params
        n_cols_alg = 0

        for j in range(d):

            # a n-1 vector recording if two betas have the same mean
            test_j = np.zeros(n-1)

            for i in range(1,n):
                all_coeff[i,j] = ols_models[i].params[j]

                z_stat = ( np.abs(ols_models[0].params[j] - ols_models[i].params[j]) / 
                          np.sqrt(np.square(ols_models[0].bse[j]) + np.square(ols_models[i].bse[j])) )
                p_value = 1 - norm.cdf(z_stat)
                if p_value >= 0.05:  #P-value ->0.05
                    test_j[i-1] = 1

            if np.sum(test_j) >= Ru*(n-1):
                aggre_level.append('dept')
                n_cols_alg += 1

            elif np.sum(test_j) <= Rl*(n-1):
                aggre_level.append('sku')
                n_cols_alg += n

            else:
                aggre_level.append('clus')
                clus_columns.append(j)
                n_cols_alg += z

        if len(clus_columns) > 0:
            X_clus = all_coeff[:, clus_columns]
            kmeans = KMeans(n_clusters=z, random_state=0).fit(X_clus)

        X_alg = np.zeros((n*obs, n_cols_alg))
        X_test = np.zeros((int(n*obs/2), n_cols_alg))

        count = 0
        for i in range(d):
            if aggre_level[i] == 'dept':
                X_alg[:,count] = data[:,i]
                X_test[:,count] = data_test[:,i]
                count += 1

            elif aggre_level[i] == 'clus':
                for j in range(z):
                    clus_items = list(np.where(kmeans.labels_ == j)[0])
                    for sku in clus_items:
                        X_alg[sku*obs:(sku+1)*obs, count] = data[sku*obs:(sku+1)*obs, i]
                        X_test[int(sku*obs/2):int((sku+1)*obs/2), count] = data_test[int(sku*obs/2):int((sku+1)*obs/2), i]


                    count += 1
            else:
                for j in range(n):
                    X_alg[j*obs:(j+1)*obs, count] = data[j*obs:(j+1)*obs, i]

                    X_test[int(j*obs/2):int((j+1)*obs/2), count] = data_test[int(j*obs/2):int((j+1)*obs/2), i]

                    count += 1


        model_0 = OLS(y, X_alg, hasconst = False)
        result = model_0.fit()
        r2_alg_t[t] = r2_score(y_test, result.predict(X_test))
        

        # model fitting - b2
        model_0 = OLS(y, data, hasconst = False)
        result = model_0.fit()
        r2_b2_t[t] = r2_score(y_test, result.predict(data_test))
        

        # model fitting - b1 Lasso
        y_pred = []
        for i in range(n):
            model_i = LassoCV(alphas=[0.01,0.1,1,10,100],max_iter=10000).fit(data[i*obs:(i+1)*obs,:], y[i*obs:(i+1)*obs])
            y_pred += list(model_i.predict(data_test[int(i*obs/2):int((i+1)*obs/2),:]))

        r2_lasso_t[t] = r2_score(y_test, np.array(y_pred))
        
        

        
        
        #other clustering
        X_clus = np.zeros((n, d))
        count = 0
        y_pred = np.zeros(int(n*obs/2))
        for i in range(n):
            X_clus[count, :] = np.mean(data[i*obs:(i+1)*obs,:], axis = 0)
            count += 1
        
        kmeans = KMeans(n_clusters=z, random_state=0).fit(X_clus)
        for i in range(z):
            clus_items = list(np.where(kmeans.labels_ == i)[0])
            base_rows = np.array(list(range(obs)))
            base_rows_test = np.array(list(range(int(obs/2))))
            rows = []
            test_rows = []
            for j in clus_items:
                rows += list(range( j*obs, (j+1)*obs ))
                test_rows += list(range( j*int(obs/2), (j+1)*int(obs/2) ))
            
            model_i = OLS(y[np.ix_(rows)], data[np.ix_(rows)], hasconst = False).fit()
            y_pred[np.ix_(test_rows)] = model_i.predict(data_test[np.ix_(test_rows)])

        r2_clus_t[t] = r2_score(y_test, y_pred)

    return np.mean(r2_alg_t), np.mean(r2_b1_t), np.mean(r2_b2_t), np.mean(r2_lasso_t), np.mean(r2_clus_t)
 

In [ ]:
list_n = [i for i in range(10,160,10)]

d=5
p=2/3
q=1/6
sigma=1
n=20
obs=10

r2_alg, r2_b1, r2_b2, r2_lasso, r2_clus = [], [], [], [],[]
for n in list_n:
    result_1, result_2, result_3, result_4,result_5 = fit_models_upd(0.9,0.6)
    r2_alg.append(result_1)
    r2_b1.append(result_2)
    r2_b2.append(result_3)
    r2_lasso.append(result_4)
    r2_clus.append(result_5)
    print("items:",n)
    print("dac:",result_1)
    print("dec",result_2)

items: 10
dac: 0.7290195171797351
dec 0.6101685189027019
items: 20
dac: 0.7461761810453673
dec 0.6281738361410864
items: 30
dac: 0.757023169662481
dec 0.650619775051135
items: 40
dac: 0.7569947606516044
dec 0.6609013275648219
items: 50
dac: 0.7535065107948333
dec 0.6233787733097677
items: 60
dac: 0.753034924663497
dec 0.6448733117721567
items: 70
dac: 0.7720804858996714
dec 0.6595030142217233
items: 80
dac: 0.7617999065417758
dec 0.6437168821307866
items: 90
dac: 0.7516541476906617
dec 0.6550049354097953
items: 100
dac: 0.7722517151317366
dec 0.6532746972601167
items: 110
dac: 0.7756364139192101
dec 0.6574566261819048
items: 120
dac: 0.767118296776763
dec 0.6334050974399046
items: 130
dac: 0.7803486753945701
dec 0.6657158592125026


In [ ]:
plt.plot(list_n, r2_alg,"--",label = 'DAC')
plt.plot(list_n, r2_b1,'',label = 'Decentralized')
plt.plot(list_n, r2_b2,linestyle=(0, (5, 2, 1, 2)), dash_capstyle='round',label = 'Centralized')
plt.plot(list_n, r2_clus,'y-o',markersize=3,label = 'Clustering')
plt.plot(list_n, r2_lasso,":",label = 'Decentralized-Lasso')


plt.legend(loc='best',frameon=False,prop={'size': 8})
plt.xlabel('n')
c1 = "2"
label = r'$OOS$ R$^{{{}}}$ '.format(c1)
plt.ylabel(label)


plt.ylim(-0.4,1.1)
plt.show()